In [1]:
import pandas as pd

train_data = pd.read_csv("stepik_data.csv")

labels = train_data['object'].to_list()
num_classes = len(set(labels))

texts = (train_data['Reviews']).to_list()
print(texts[0])

Лучший курс для подготовки к ЕГЭ по математике с отличным учителем ever Учёба с Тимуром = СОТОЧКА баллов за экзамен + отлично проведённое время


In [2]:
print(labels)
print(texts)

[1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 0, 2, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 

In [4]:
import torch
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

# Загрузка данных
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Функция для тренировки модели
def train_model(model, train_dataloader, val_dataloader, optimizer, epochs):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for batch in tqdm(train_dataloader, total=len(train_dataloader), desc=f'Epoch {epoch + 1}/{epochs}', unit='batch'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_dataloader)

        print(f'Training Loss: {avg_train_loss}')

        # Валидация
        model.eval()
        val_preds = []
        val_labels = []
        for batch in tqdm(val_dataloader, total=len(val_dataloader), desc=f'Validating Epoch {epoch + 1}/{epochs}', unit='batch'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].numpy()

            with torch.no_grad():
                outputs = model(input_ids, attention_mask=attention_mask)
                logits = outputs.logits
                preds = np.argmax(logits.detach().cpu().numpy(), axis=1)
                val_preds.extend(preds)
                val_labels.extend(labels)

        val_f1 = f1_score(val_labels, val_preds, average='macro')
        print(f'Validation F1-Score: {val_f1}')

# Загрузка предобученного RuBERT
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model = BertForSequenceClassification.from_pretrained('DeepPavlov/rubert-base-cased', num_labels=num_classes)

# Параметры
max_len = 128
batch_size = 32
epochs = 3
learning_rate = 2e-5

# Подготовка данных
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

train_dataset = CustomDataset(train_texts, train_labels, tokenizer, max_len)
val_dataset = CustomDataset(val_texts, val_labels, tokenizer, max_len)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

# Инициализация оптимизатора
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Обучение модели
train_model(model, train_dataloader, val_dataloader, optimizer, epochs)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/3: 100%|██████████| 143/143 [10:30<00:00,  4.41s/batch]


Training Loss: 0.41930482906478267


Validating Epoch 1/3: 100%|██████████| 36/36 [00:30<00:00,  1.16batch/s]


Validation F1-Score: 0.3137436372049977


Epoch 2/3: 100%|██████████| 143/143 [09:56<00:00,  4.17s/batch]


...

Validation F1-Score: 0.3137436372049977


Epoch 3/3: 100%|██████████| 143/143 [10:00<00:00,  4.20s/batch]


Training Loss: 0.33744627130510924


Validating Epoch 3/3: 100%|██████████| 36/36 [00:31<00:00,  1.15batch/s]

Validation F1-Score: 0.3423095339254523


In [5]:
test_data = pd.read_csv("train_Feedback/train_data.csv")
test_labels = test_data['object']
test_texts = (test_data['question_2'] + test_data['question_3'] + test_data['question_4'] + test_data['question_5']).to_list()

In [6]:
test_dataset = CustomDataset(test_texts, test_labels, tokenizer, max_len)

test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Валидация
model.eval()
val_preds = []
val_labels = []
for batch in tqdm(test_dataloader, total=len(test_dataloader)):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['label'].numpy()

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = np.argmax(logits.detach().cpu().numpy(), axis=1)
        val_preds.extend(preds)
        val_labels.extend(labels)

val_f1 = f1_score(val_labels, val_preds, average='micro')
print(f'Test F1-Score: {val_f1}')

100%|██████████| 4/4 [00:02<00:00,  1.40it/s]

Test F1-Score: 0.3106796116504854


In [7]:
model_path = "object/model.bin"

# Сохранение дообученной модели в bin файл
model.save_pretrained(model_path)